<a href="https://colab.research.google.com/github/advaitha/D2AI/blob/main/concise_implementation_linear_regression_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
def synthetic_data(w , b, num_examples):
  X = tf.zeros((num_examples, w.shape[0]))
  X += tf.random.normal(shape=X.shape)
  y = tf.matmul(X,tf.reshape(w,(-1,1))) + b
  y += tf.random.normal(shape = y.shape, stddev=0.01)
  y = tf.reshape(y,(-1,1))
  return X, y

In [5]:
import numpy as np
import tensorflow as tf
from d2l import tensorflow as d2l

true_w = tf.constant([2,-3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

In [6]:
# Load the dataset
def load_array(data_arrays, batch_size, is_train=True):
  dataset = tf.data.Dataset.from_tensor_slices(data_arrays)
  if is_train:
    dataset = dataset.shuffle(buffer_size=1000)
  dataset = dataset.batch(batch_size)
  return dataset

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [7]:
next(iter(data_iter))

(<tf.Tensor: shape=(10, 2), dtype=float32, numpy=
 array([[-0.28774443, -0.20485936],
        [ 1.003616  ,  0.16492677],
        [-0.5845998 ,  2.876937  ],
        [ 0.11662029,  0.54943293],
        [-2.2438135 , -1.1620028 ],
        [ 1.4517365 , -0.6686997 ],
        [-1.0269731 ,  1.3693098 ],
        [-0.75879145, -0.04185718],
        [-0.9673758 , -0.23666677],
        [ 0.02166146, -0.35773194]], dtype=float32)>,
 <tf.Tensor: shape=(10, 1), dtype=float32, numpy=
 array([[ 4.3197556],
        [ 5.641314 ],
        [-6.7631793],
        [ 2.5761807],
        [ 3.6753256],
        [ 9.378376 ],
        [-2.5116124],
        [ 2.8393009],
        [ 3.0700946],
        [ 5.469989 ]], dtype=float32)>)

In [8]:
# No need to specify input shape to keras
net = tf.keras.Sequential()
net.add(tf.keras.layers.Dense(1))

In [9]:
# Initialize Model Parameters - Normal distribution with mean 0 and std. deviation 0.01
initializer = tf.initializers.RandomNormal(stddev=0.01)
net = tf.keras.Sequential()
net.add(tf.keras.layers.Dense(1, kernel_initializer=initializer))

In [10]:
# loss function
loss = tf.keras.losses.MeanSquaredError()

In [11]:
trainer = tf.keras.optimizers.SGD(learning_rate=0.03)

In [13]:
num_epochs = 3
for epoch in range(num_epochs):
  for X,y in data_iter:
    with tf.GradientTape() as tape:
      l = loss(net(X,training = True),y)
    grads = tape.gradient(l,net.trainable_variables)
    trainer.apply_gradients(zip(grads, net.trainable_variables))
  l = loss(net(features),labels)
  print(f'epoch {epoch + 1} , loss {l:f}')

epoch 1 , loss 0.000203
epoch 2 , loss 0.000087
epoch 3 , loss 0.000087


In [14]:
# Access the layer and get the layer's weight
w = net.get_weights()[0]
print('error in estimating w', true_w - tf.reshape(w,true_w.shape))
b = net.get_weights()[1]
print('error in estimating b',true_b - b)

error in estimating w tf.Tensor([0.00017917 0.00034642], shape=(2,), dtype=float32)
error in estimating b [-9.059906e-06]
